# Importing Libraries/Connecting to GDrive

In [3]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Mounted at /gdrive
/gdrive


# Exploratory Data Analysis

In [4]:
trainfile = r'/gdrive/My Drive/CIS 508/Assignment 2/Insurance Fraud - TRAIN.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS 508/Assignment 2/Insurance Fraud - TEST.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)



(2999, 32)
(12918, 32)


In [5]:
trainData.head(5)

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [6]:
trainData.info()
trainData.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MONTH                 2999 non-null   object
 1   WEEKOFMONTH           2999 non-null   int64 
 2   DAYOFWEEK             2999 non-null   object
 3   MAKE                  2999 non-null   object
 4   ACCIDENTAREA          2999 non-null   object
 5   DAYOFWEEKCLAIMED      2999 non-null   object
 6   MONTHCLAIMED          2999 non-null   object
 7   WEEKOFMONTHCLAIMED    2999 non-null   int64 
 8   SEX                   2999 non-null   object
 9   MARITALSTATUS         2999 non-null   object
 10  AGE                   2999 non-null   int64 
 11  FAULT                 2999 non-null   object
 12  POLICYTYPE            2999 non-null   object
 13  VEHICLECATEGORY       2999 non-null   object
 14  VEHICLEPRICE          2999 non-null   object
 15  REPNUMBER             2999 non-null   

,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR
count,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000
mean,2.781260,2.671224,40.055352,8.511837,407.302434,2.496832,1995.114038
std,1.286055,1.261614,13.497026,4.601437,41.847258,1.118365,0.606007
min,1.000000,1.000000,0.000000,1.000000,300.000000,1.000000,1994.000000
25%,2.000000,2.000000,31.000000,4.000000,400.000000,1.000000,1995.000000
50%,3.000000,3.000000,38.000000,9.000000,400.000000,2.000000,1995.000000
75%,4.000000,4.000000,49.000000,12.000000,400.000000,3.000000,1995.000000
max,5.000000,5.000000,80.000000,16.000000,700.000000,4.000000,1996.000000


In [7]:
trainData.isna().sum()

MONTH                   0
WEEKOFMONTH             0
DAYOFWEEK               0
MAKE                    0
ACCIDENTAREA            0
DAYOFWEEKCLAIMED        0
MONTHCLAIMED            0
WEEKOFMONTHCLAIMED      0
SEX                     0
MARITALSTATUS           0
AGE                     0
FAULT                   0
POLICYTYPE              0
VEHICLECATEGORY         0
VEHICLEPRICE            0
REPNUMBER               0
DEDUCTIBLE              0
DRIVERRATING            0
DAYS_POLICY_ACCIDENT    0
DAYS_POLICY_CLAIM       0
PASTNUMBEROFCLAIMS      0
AGEOFVEHICLE            0
AGEOFPOLICYHOLDER       0
POLICEREPORTFILED       0
WITNESSPRESENT          0
AGENTTYPE               0
NUMBEROFSUPPLIMENTS     0
ADDRESSCHANGE_CLAIM     0
NUMBEROFCARS            0
YEAR                    0
BASEPOLICY              0
FRAUDFOUND              0
dtype: int64

In [8]:
#List of Categorical Features
categoricalFeatures = ["MONTH", "DAYOFWEEK", "MAKE", "ACCIDENTAREA", "DAYOFWEEKCLAIMED", \
                        "MONTHCLAIMED", "SEX", "MARITALSTATUS", "AGE", "FAULT", \
                        "POLICYTYPE", "VEHICLECATEGORY", "VEHICLEPRICE", \
                        "DAYS_POLICY_ACCIDENT", "DAYS_POLICY_CLAIM", \
                        "PASTNUMBEROFCLAIMS", "AGEOFVEHICLE", "AGEOFPOLICYHOLDER",\
                        "POLICEREPORTFILED", "WITNESSPRESENT", "AGENTTYPE", "NUMBEROFSUPPLIMENTS", \
                        "ADDRESSCHANGE_CLAIM", "NUMBEROFCARS", "BASEPOLICY", "WEEKOFMONTH", "WEEKOFMONTHCLAIMED",
                       "REPNUMBER", "DEDUCTIBLE", "DRIVERRATING"]

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData,testData], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
trainData = combined_Data.xs(0)
testData = combined_Data.xs(1)

y_train = trainData['FRAUDFOUND']
X_train = trainData.drop(['FRAUDFOUND'], axis=1) #extracting training data without the target column
y_test = testData['FRAUDFOUND']
X_test = testData.drop(['FRAUDFOUND'], axis=1) #extracting testing data without the target column


# Decision Tree Modeling

In [88]:
#Decision Tree Classifier ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (testset) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))



accuracy Score (testset) for Decision Tree:0.896037
Confusion Matrix for Decision Tree
[[11131  1289]
 [   54   444]]


In [89]:
print(clf.feature_importances_)

[0.11206823 0.00144544 0.00571293 0.00632343 0.00908239 0.0065342
 0.00412983 0.00240907 0.01053643 0.         0.0022878  0.00945385
 0.00501074 0.00129057 0.00354133 0.01501137 0.00681996 0.00849494
 0.00644827 0.00553397 0.00673436 0.         0.0049558  0.
 0.         0.00313179 0.00252952 0.         0.         0.00760311
 0.         0.0024779  0.00221021 0.01235809 0.         0.
 0.00211039 0.00124738 0.00431112 0.         0.00192725 0.00931892
 0.00822524 0.00664334 0.         0.00372381 0.01057236 0.01165397
 0.01012311 0.00754569 0.00282518 0.00192725 0.00511647 0.00371553
 0.00690318 0.00199412 0.00816118 0.00660497 0.00293385 0.00068566
 0.0023127  0.00393775 0.         0.         0.00713084 0.
 0.00134423 0.         0.         0.         0.         0.
 0.         0.00246382 0.         0.00371589 0.         0.00476447
 0.00813756 0.00055676 0.00391746 0.00867476 0.00187392 0.00363245
 0.0015418  0.         0.0070284  0.         0.00060931 0.
 0.00085793 0.         0.         0.

###Decision Tree Hyperparameter Tuning 1




In [90]:
#Hyperparameter tuning done for decision tree classifier

#do random search
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(100,1000,100),'max_depth': range(15,50,15),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10,cv=10)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 700, 'max_depth': 15, 'criterion': 'entropy'}


In [91]:
#Now do grid search
print("GridSearchCV-Decision tree")
parameters2={'min_samples_leaf' : range(10,50,10),'criterion':['gini','entropy'], 'splitter':['best','random']}
clf_grid = GridSearchCV(clf, parameters2, cv=5, verbose=1)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

GridSearchCV-Decision tree
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'criterion': 'entropy', 'min_samples_leaf': 40, 'splitter': 'random'}


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    2.9s finished


In [92]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)




In [93]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))


accuracy Score (testset) after hypertuning randomized search for Decision Tree:0.961449
accuracy Score (testset) after hypertuning grid search for Decision Tree:0.893482
Confusion Matrix after hypertuning for Decision Tree
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.96      1.00      0.98     12420
         Yes       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918



In [98]:
#cross validation using decision tree model
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=7, 
                               verbose=2, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[0.5 0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s finished


###Decision Tree Hyperparameter Tuning 2




In [99]:
#Hyperparameter tuning done for decision tree classifier

#do random search
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(40,200,40),'max_depth': range(40,200,25),'criterion':['entropy'], 'max_leaf_nodes' : range(10,100,10)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=4,cv=6)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 40, 'max_leaf_nodes': 90, 'max_depth': 65, 'criterion': 'entropy'}


In [100]:
#Now do grid search
print("GridSearchCV-Decision tree")
parameters2={'min_samples_leaf' : range(30,200,30),'criterion':['gini','entropy'], 'splitter':['best']}
clf_grid = GridSearchCV(clf, parameters2, cv=5, verbose=3, n_jobs=2)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

GridSearchCV-Decision tree
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


{'criterion': 'entropy', 'min_samples_leaf': 60, 'splitter': 'best'}


[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:    1.7s finished


In [101]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)


In [102]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

accuracy Score (testset) after hypertuning randomized search for Decision Tree:0.893482
accuracy Score (testset) after hypertuning grid search for Decision Tree:0.882877
Confusion Matrix after hypertuning for Decision Tree
[[11378  1042]
 [  334   164]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.92      0.94     12420
         Yes       0.14      0.33      0.19       498

    accuracy                           0.89     12918
   macro avg       0.55      0.62      0.57     12918
weighted avg       0.94      0.89      0.91     12918



In [103]:
#cross validation using decision tree model
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, 
                               verbose=8, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


[CV]  ................................................................
[CV] .................................... , score=0.890, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.762, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.780, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.675, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.588, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.500, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.500, total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s


[CV] .................................... , score=0.615, total=   0.0s
[0.89038462 0.76153846 0.77980769 0.675      0.5875     0.5
 0.5        0.5        0.55       0.61538462]




[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished


###Decision Tree Hyperparameter Tuning 3


In [104]:
#Hyperparameter tuning done for decision tree classifier

#do random search
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(25,100,25),'criterion':['entropy'], 'max_leaf_nodes' : range( 20,100,20)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=20,cv=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

RandomizedSearchCV-Decision tree
{'min_samples_split': 70, 'max_leaf_nodes': 80, 'max_depth': 75, 'criterion': 'entropy'}


In [105]:
#Now do grid search
print("GridSearchCV-Decision tree")
parameters2={'min_samples_leaf' : range(5,100,50),'criterion':['gini','entropy'], 'splitter':['random'], 'max_leaf_nodes' : range( 20,100,20)}
clf_grid = GridSearchCV(clf, parameters2, cv=2)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_leaf_nodes': 20, 'min_samples_leaf': 5, 'splitter': 'random'}


In [106]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)

In [107]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

accuracy Score (testset) after hypertuning randomized search for Decision Tree:0.903081
accuracy Score (testset) after hypertuning grid search for Decision Tree:0.929246
Confusion Matrix after hypertuning for Decision Tree
[[11422   998]
 [  254   244]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.98      0.92      0.95     12420
         Yes       0.20      0.49      0.28       498

    accuracy                           0.90     12918
   macro avg       0.59      0.70      0.61     12918
weighted avg       0.95      0.90      0.92     12918



In [108]:
#cross validation using decision tree model
clf_cv_score = cross_val_score(clf, X_train, y_train)
print(clf_cv_score)
print('\n')

[0.89833333 0.87333333 0.86       0.86166667 0.82136895]




# Random Forest Modeling

In [9]:
#Normal randomforest==============================================================================
#=================================================================================================
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (testset) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))


accuracy Score (testset) for RandomForest:0.965707
Confusion Matrix for Random Forest:
[[12056   364]
 [   79   419]]


###Random Forest Hyperparameter Tuning 1

In [10]:
#Hyperparameter tuning done for random forest classifier

#do random search
print("RandomizedSearchCV-Random Forest")
r_parameters={'min_samples_split' : range(10,100,10),'max_depth': range(25,100,25),'criterion':['entropy'], 'max_leaf_nodes' : range( 20,100,20)}
rfc_random = RandomizedSearchCV(rfc,r_parameters,n_iter=5,cv=10)
rfc_random.fit(X_train, y_train)
grid_parmR=rfc_random.best_params_
print(grid_parmR)

RandomizedSearchCV-Random Forest
{'min_samples_split': 40, 'max_leaf_nodes': 80, 'max_depth': 75, 'criterion': 'entropy'}


In [11]:
#Now do grid search
print("GridSearchCV-Decision tree")
r_parameters2={'n_estimators': [5, 15, 30, 50],'criterion': ['gini', 'entropy']}
rfc_grid = GridSearchCV(rfc, r_parameters2, n_jobs=-1)
rfc_grid.fit(X_train, y_train)
grid_parmR1=rfc_grid.best_params_
print(grid_parmR1)

GridSearchCV-Decision tree
{'criterion': 'entropy', 'n_estimators': 15}


In [12]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
#Construct Random Forest using the best parameters
rfc = RandomForestClassifier(**grid_parmR)
rfcr = RandomForestClassifier(**grid_parmR1)

rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfcr.fit(X_train,y_train)
rfcr_predict = rfcr.predict(X_test)

In [25]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Random Forest:{0:6f}".format(rfcr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

accuracy Score (testset) after hypertuning randomized search for Random Forest:0.961449
accuracy Score (testset) after hypertuning grid search for Random Forest:0.963617
Confusion Matrix after hypertuning for Random Forest
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.96      1.00      0.98     12420
         Yes       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918



In [14]:
#cross validation using random forest model
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=4, 
                               scoring="balanced_accuracy", error_score=5)
print(rfc_cv_score)
print('\n')

[0.70730769 0.515      0.5        0.52525253]




###Random Forest Hyperparameter Tuning 2




In [27]:
#Hyperparameter tuning done for random forest classifier

#do random search
print("RandomizedSearchCV-Random Forest")
r_parameters={'min_samples_split' : range(85,570,100),'max_depth': range(15,80,15),'criterion':['gini'], 'max_leaf_nodes' : range( 200,1000,200)}
rfc_random = RandomizedSearchCV(rfc,r_parameters,n_iter=10,cv=2)
rfc_random.fit(X_train, y_train)
grid_parmR=rfc_random.best_params_
print(grid_parmR)

RandomizedSearchCV-Random Forest
{'min_samples_split': 85, 'max_leaf_nodes': 600, 'max_depth': 30, 'criterion': 'gini'}


In [28]:
#Now do grid search
print("GridSearchCV-Decision tree")
r_parameters2={'n_estimators': [25, 55, 10, 20],'criterion': ['gini', 'entropy']}
rfc_grid = GridSearchCV(rfc, r_parameters2, n_jobs=5, verbose=5)
rfc_grid.fit(X_train, y_train)
grid_parmR1=rfc_grid.best_params_
print(grid_parmR1)

GridSearchCV-Decision tree
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    4.5s


{'criterion': 'gini', 'n_estimators': 25}


[Parallel(n_jobs=5)]: Done  40 out of  40 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=5)]: Done  40 out of  40 | elapsed:    6.6s finished


In [29]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
#Construct Random Forest using the best parameters
rfc = RandomForestClassifier(**grid_parmR)
rfcr = RandomForestClassifier(**grid_parmR1)

rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfcr.fit(X_train,y_train)
rfcr_predict = rfcr.predict(X_test)

In [30]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Random Forest:{0:6f}".format(rfcr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

accuracy Score (testset) after hypertuning randomized search for Random Forest:0.954947
accuracy Score (testset) after hypertuning grid search for Random Forest:0.960675
Confusion Matrix after hypertuning for Random Forest
[[12258   162]
 [  420    78]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.99      0.98     12420
         Yes       0.33      0.16      0.21       498

    accuracy                           0.95     12918
   macro avg       0.65      0.57      0.59     12918
weighted avg       0.94      0.95      0.95     12918



In [31]:
#cross validation using random forest model
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=5, 
                               scoring="balanced_accuracy", 
                               n_jobs=3, error_score=3)
print(rfc_cv_score)
print('\n')

[0.53461538 0.5125     0.5125     0.5        0.51265823]




###Random Forest Hyperparameter Tuning 3


In [32]:
#Hyperparameter tuning done for random forest classifier

#do random search
print("RandomizedSearchCV-Random Forest")
r_parameters={'min_samples_split' : range(200,800,150),'max_depth': range(10,200,80),'criterion':['entropy'], 'max_leaf_nodes' : range( 5,90,2)}
rfc_random = RandomizedSearchCV(rfc,r_parameters,n_iter=2,cv=3, verbose=1)
rfc_random.fit(X_train, y_train)
grid_parmR=rfc_random.best_params_
print(grid_parmR)

RandomizedSearchCV-Random Forest
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.5s finished


{'min_samples_split': 500, 'max_leaf_nodes': 85, 'max_depth': 170, 'criterion': 'entropy'}


In [33]:
#Now do grid search
print("GridSearchCV-Decision tree")
r_parameters2={'n_estimators': [95, 10, 120, 230],'criterion': ['gini', 'entropy'], 'class_weight':['balanced', 'balanced_subsample']}
rfc_grid = GridSearchCV(rfc, r_parameters2, n_jobs=3, verbose=1, error_score=-2)
rfc_grid.fit(X_train, y_train)
grid_parmR1=rfc_grid.best_params_
print(grid_parmR1)

GridSearchCV-Decision tree
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   15.2s
[Parallel(n_jobs=3)]: Done  80 out of  80 | elapsed:   31.0s finished


{'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'n_estimators': 95}


In [34]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
#Construct Random Forest using the best parameters
rfc = RandomForestClassifier(**grid_parmR)
rfcr = RandomForestClassifier(**grid_parmR1)

rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfcr.fit(X_train,y_train)
rfcr_predict = rfcr.predict(X_test)

In [35]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))

print("accuracy Score (testset) after hypertuning grid search for Random Forest:{0:6f}".format(rfcr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

accuracy Score (testset) after hypertuning randomized search for Random Forest:0.961449
accuracy Score (testset) after hypertuning grid search for Random Forest:0.961139
Confusion Matrix after hypertuning for Random Forest
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.96      1.00      0.98     12420
         Yes       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918



In [36]:
#cross validation using random forest model
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, 
                               scoring="balanced_accuracy", n_jobs=9, 
                               error_score=-1, verbose=3)
print(rfc_cv_score)
print('\n')

[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
[Parallel(n_jobs=9)]: Done   5 out of  10 | elapsed:    5.7s remaining:    5.7s


[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]




[Parallel(n_jobs=9)]: Done  10 out of  10 | elapsed:    6.1s finished
